In [1]:
# Set up logging

import logging

logging.basicConfig(
    format="[%(asctime)s %(levelname)s %(threadName)s %(name)s:%(funcName)s:%(lineno)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S%z",
)
logging.getLogger("council").setLevel(logging.INFO)

In [2]:
# Council imports

from council.runners import Budget
from council.contexts import AgentContext, ChatHistory
from council.agents import Agent
from council.chains import Chain
from council.llm.openai_llm_configuration import OpenAILLMConfiguration
from council.llm.openai_llm import OpenAILLM

from skills import SectionWriterSkill, OutlineWriterSkill
from controller import WritingAssistantController
from evaluator import BasicEvaluatorWithSource

In [3]:
# Read environment variables and initialize OpenAI LLM instance

import dotenv
dotenv.load_dotenv()
openai_llm = OpenAILLM(config=OpenAILLMConfiguration.from_env())

In [4]:
# Create Skills

outline_skill = OutlineWriterSkill(openai_llm)
writing_skill = SectionWriterSkill(openai_llm)

In [5]:
# Create Chains

outline_chain = Chain(
    name="Outline Writer",
    description="Write or revise the outline (i.e. section headers) of a research article in markdown format. Only use this chain when the ARTICLE OUTLINE is missing or incomplete.",
    runners=[outline_skill]
)

writer_chain = Chain(
    name="Article Writer",
    description="Write or revise specific section bodies of a research article in markdown format. Use this chain to write the main research article content.",
    runners=[writing_skill]
)

In [6]:
# Create Controller

controller = WritingAssistantController(
    openai_llm,
    top_k_execution_plan=3
)

In [7]:
# Initialize Agent

chat_history = ChatHistory()
run_context = AgentContext(chat_history)
agent = Agent(controller, [outline_chain, writer_chain], BasicEvaluatorWithSource())

In [8]:
# Run Agent

chat_history.add_user_message("What is the financial performance of Microsoft?")
result = agent.execute(run_context, Budget(1200))

[2023-07-25 15:23:06-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-25 15:23:06-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="1"
[2023-07-25 15:23:12-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="Article Writer" execution_unit="Article Writer: Start with writing the Introduction and Company Overview sections of the article, providing a brief about Microsoft and its business operations."
[2023-07-25 15:23:12-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="SectionWriterSkill"
[2023-07-25 15:23:34-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="SectionWriterSkill" skill_message="I've written or edited the article and placed it in the 'data' field."
[2023-07-25 15:23:34-0400 INFO MainThread council.agents

In [9]:
print(result.messages[-1].message.message)

"# Introduction

Microsoft Corporation is a multinational technology company that has significantly shaped the software industry. With its headquarters in Redmond, Washington, Microsoft is renowned for its software products like the Microsoft Windows operating systems, the Microsoft Office suite, and the Internet Explorer and Edge web browsers. 

# Company Overview

Founded by Bill Gates and Paul Allen in 1975, Microsoft has grown to become one of the largest and most influential tech companies in the world. The company's primary business is centered around developing, manufacturing, licensing, supporting, and selling computer software, consumer electronics, personal computers, and related services. 

Microsoft's most notable products include the Windows line of operating systems, the Office Suite of productivity software, and the Azure cloud computing platform. The company also has a significant presence in the hardware market with products like the Xbox video game consoles and the Su

In [10]:
chat_history.add_user_message("Can you please add a section about environmental priorities? ")
result = agent.execute(run_context, Budget(600))

[2023-07-25 15:25:54-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-25 15:25:54-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="3"
[2023-07-25 15:26:01-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="Article Writer" execution_unit="Article Writer: Write a new section titled "Environmental Priorities" discussing Microsoft's environmental initiatives, sustainability goals, and how these priorities might impact their financial performance."
[2023-07-25 15:26:01-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="SectionWriterSkill"
[2023-07-25 15:26:46-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="SectionWriterSkill" skill_message="I've written or edited the article and placed it in the 'data' field."
[2023-07

In [11]:
print(result.messages[-1].message.message)

"# Introduction

Microsoft Corporation is a multinational technology company that has significantly shaped the software industry. With its headquarters in Redmond, Washington, Microsoft is renowned for its software products like the Microsoft Windows operating systems, the Microsoft Office suite, and the Internet Explorer and Edge web browsers. 

# Company Overview

Founded by Bill Gates and Paul Allen in 1975, Microsoft has grown to become one of the largest and most influential tech companies in the world. The company's primary business is centered around developing, manufacturing, licensing, supporting, and selling computer software, consumer electronics, personal computers, and related services. 

Microsoft's most notable products include the Windows line of operating systems, the Office Suite of productivity software, and the Azure cloud computing platform. The company also has a significant presence in the hardware market with products like the Xbox video game consoles and the Su

In [13]:
chat_history.add_user_message("Can you please expand the article in every section?")
result = agent.execute(run_context, Budget(1200))

[2023-07-25 15:37:20-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-25 15:37:20-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="4"
[2023-07-25 15:37:25-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="Article Writer" execution_unit="Article Writer: Add a new section on "Environmental Priorities" as per user request."
[2023-07-25 15:37:25-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="SectionWriterSkill"
[2023-07-25 15:38:12-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="SectionWriterSkill" skill_message="I've written or edited the article and placed it in the 'data' field."
[2023-07-25 15:38:12-0400 INFO MainThread council.agents.agent:execute:76] message="chain execution ended" chain="Article Writer" ex

In [14]:
print(result.messages[-1].message.message)

"# Introduction

Microsoft Corporation is a multinational technology company that has significantly shaped the software industry. With its headquarters in Redmond, Washington, Microsoft is renowned for its software products like the Microsoft Windows operating systems, the Microsoft Office suite, and the Internet Explorer and Edge web browsers. This article provides a comprehensive analysis of Microsoft's financial performance, including revenue, profitability, liquidity, and solvency. It also discusses the company's environmental priorities and their potential impact on its financial performance.

# Company Overview

Founded by Bill Gates and Paul Allen in 1975, Microsoft has grown to become one of the largest and most influential tech companies in the world. The company's primary business is centered around developing, manufacturing, licensing, supporting, and selling computer software, consumer electronics, personal computers, and related services. 

Microsoft's most notable product